In [1]:
import pandas as pd
import cv2
import os
from PIL import Image
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
img_dir = '../cropped_img/II/'

img_size = 640, 320

img_ds = []
label = []

for file in os.listdir(img_dir):
    img = cv2.imread(img_dir+file)
    H, W = img.shape[:2]
    rs_img = cv2.resize(img,img_size)
    label.append(file.split('_')[-1].split('.')[0])
    img_ds.append(rs_img)

    

In [3]:
len(img_ds)

633

In [4]:
len(label)

633

In [5]:
pd.Series(label).value_counts()

fa        500
normal    133
Name: count, dtype: int64

In [6]:
y = np.array([1 if x == 'fa' else 0 for x in label])
dfy = pd.DataFrame({'y':y})

In [7]:
img_ds = np.array(img_ds)

In [8]:
img_ds.shape

(633, 320, 640, 3)

In [13]:
# fa_idx = dfy[dfy['y'] == 1].head(150).index.tolist()
# normal_idx = dfy[dfy['y'] == 0].index.tolist()

# fa_idx + normal_idx

[0,
 2,
 4,
 6,
 8,
 10,
 12,
 14,
 16,
 18,
 20,
 22,
 24,
 26,
 28,
 30,
 32,
 34,
 36,
 38,
 40,
 42,
 44,
 46,
 48,
 50,
 52,
 54,
 56,
 58,
 60,
 62,
 64,
 66,
 68,
 70,
 72,
 74,
 76,
 78,
 80,
 82,
 84,
 86,
 88,
 90,
 92,
 94,
 96,
 98,
 100,
 102,
 104,
 106,
 108,
 110,
 112,
 114,
 116,
 118,
 120,
 122,
 124,
 126,
 128,
 130,
 132,
 134,
 136,
 138,
 140,
 142,
 144,
 146,
 148,
 150,
 152,
 154,
 156,
 158,
 160,
 162,
 164,
 166,
 168,
 170,
 172,
 174,
 176,
 178,
 180,
 182,
 184,
 186,
 188,
 190,
 192,
 194,
 196,
 198,
 200,
 202,
 204,
 206,
 208,
 210,
 212,
 214,
 216,
 218,
 220,
 222,
 224,
 226,
 228,
 230,
 232,
 234,
 236,
 238,
 240,
 242,
 244,
 246,
 248,
 250,
 252,
 254,
 256,
 258,
 260,
 262,
 264,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 282,
 1,
 3,
 5,
 7,
 9,
 11,
 13,
 15,
 17,
 19,
 21,
 23,
 25,
 27,
 29,
 31,
 33,
 35,
 37,
 39,
 41,
 43,
 45,
 47,
 49,
 51,
 53,
 55,
 57,
 59,
 61,
 63,


In [14]:
X = img_ds[:283,:,:,:]
y = y[:283]

X.shape, y.shape

((283, 320, 640, 3), (283,))

In [15]:
pd.Series(y).value_counts()

1    150
0    133
Name: count, dtype: int64

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Further split the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [17]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the model architecture
model = Sequential()

# First convolutional layer with 32 filters, a 3x3 kernel, and ReLU activation
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(320, 640, 3)))

# Max pooling layer with a 2x2 pool size
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolutional layer with 64 filters, a 3x3 kernel, and ReLU activation
model.add(Conv2D(64, (3, 3), activation='relu'))

# Max pooling layer with a 2x2 pool size
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Fully connected layer with 512 units and ReLU activation
model.add(Dense(512, activation='relu'))

# Dropout layer with a rate of 0.5 to reduce overfitting
model.add(Dropout(0.5))

# Output layer with a sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

In [18]:
# Compile the model with binary cross-entropy loss, Adam optimizer, and accuracy metric
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model with the specified batch size and number of epochs
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
6/6 [==============================] - 147s 10s/step - loss: 16015.1699 - accuracy: 0.4944 - val_loss: 1871.6849 - val_accuracy: 0.5000
Epoch 2/10
6/6 [==============================] - 54s 9s/step - loss: 689.7623 - accuracy: 0.4833 - val_loss: 13.5739 - val_accuracy: 0.5000
Epoch 3/10
6/6 [==============================] - 52s 9s/step - loss: 47.3683 - accuracy: 0.5167 - val_loss: 0.7428 - val_accuracy: 0.5217
Epoch 4/10
6/6 [==============================] - 53s 9s/step - loss: 1.2502 - accuracy: 0.5500 - val_loss: 0.6890 - val_accuracy: 0.5435
Epoch 5/10
6/6 [==============================] - 51s 9s/step - loss: 0.6706 - accuracy: 0.5833 - val_loss: 0.6923 - val_accuracy: 0.5435
Epoch 6/10
6/6 [==============================] - 51s 9s/step - loss: 0.6735 - accuracy: 0.5778 - val_loss: 0.6919 - val_accuracy: 0.5435
Epoch 7/10
6/6 [==============================] - 52s 9s/step - loss: 0.6775 - accuracy: 0.5833 - val_loss: 0.6911 - val_accuracy: 0.5435
Epoch 8/10
6/6 [=====

In [19]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

2/2 - 4s - loss: 0.6931 - accuracy: 0.4737 - 4s/epoch - 2s/step


In [20]:
sum(y_test),len(y_test)

(30, 57)

In [21]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

2/2 - 4s - loss: 0.6931 - accuracy: 0.4737 - 4s/epoch - 2s/step


In [22]:
pred = model.predict(X_test)

2/2 [==============================] - 4s 2s/step


In [23]:
pred_l = np.round(pred)

list(zip(pred_l.flatten().tolist(),y_test))

[(0.0, 0),
 (0.0, 1),
 (0.0, 1),
 (0.0, 1),
 (1.0, 0),
 (0.0, 0),
 (0.0, 1),
 (0.0, 1),
 (0.0, 1),
 (0.0, 0),
 (0.0, 0),
 (0.0, 1),
 (0.0, 1),
 (0.0, 0),
 (0.0, 1),
 (0.0, 1),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 1),
 (1.0, 1),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 1),
 (0.0, 1),
 (0.0, 1),
 (0.0, 0),
 (0.0, 0),
 (0.0, 1),
 (0.0, 0),
 (0.0, 1),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 1),
 (0.0, 1),
 (0.0, 1),
 (0.0, 1),
 (0.0, 1),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 1),
 (0.0, 0),
 (0.0, 1),
 (0.0, 1),
 (0.0, 0),
 (0.0, 0),
 (0.0, 1),
 (0.0, 1),
 (0.0, 1),
 (0.0, 0),
 (0.0, 1),
 (0.0, 1)]

In [24]:
model.save('app/model/model_II_v2')

INFO:tensorflow:Assets written to: app/model/model_II_v2\assets


INFO:tensorflow:Assets written to: app/model/model_II_v2\assets
